In [1]:
#http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [7]:
import torch
from torch.autograd import Variable
import torch.nn as nn

import torch.nn.functional as F
#http://pytorch.org/docs/master/nn.html#torch-nn-functional


print ("torch.__version__:", torch.__version__)

#https://github.com/torch/nn


torch.__version__: 0.3.1.post3


In [8]:
class Net(nn.Module):
    #http://pytorch.org/docs/master/nn.html#torch.nn.Module
    #Base class for all neural network modules.

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel

        #http://pytorch.org/docs/master/nn.html#torch.nn.Conv2d
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
        #nb: The parameters kernel_size, stride, padding, dilation can either be:
        #    a single int – in which case the same value is used for the height and width dimension
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        
        #http://pytorch.org/docs/master/nn.html#linear-layers
        #torch.nn.Linear(in_features, out_features, bias=True)
        #in_features, out_features, bias=True 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window using a 2x2 kernel_size
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #http://pytorch.org/docs/master/nn.html#torch.nn.functional.max_pool1d
        #torch.nn.functional.max_pool2d(input, kernel_size, stride=None, padding=0, dilation=1, ceil_mode=False, return_indices=False)
        #F.relu = http://pytorch.org/docs/master/nn.html#torch.nn.functional.relu
        #torch.nn.functional.relu6(input, inplace=False) 
        #Applies the element-wise function ReLU6(x)
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #http://pytorch.org/docs/master/nn.html#torch.nn.functional.max_pool2d
        #
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [27]:

net = Net()
print(net)
print (type(net))

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
<class '__main__.Net'>


In [13]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
#print (params)

10
torch.Size([6, 1, 5, 5])


In [16]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print("out:\n", out)
print ("type(out):\n", type(out))

out:
 Variable containing:
 0.1294 -0.0213 -0.1273  0.0833 -0.1283 -0.0328 -0.0429 -0.0398 -0.0802 -0.0493
[torch.FloatTensor of size 1x10]

type(out):
 <class 'torch.autograd.variable.Variable'>


In [17]:
#zero the gradient buffers of all parameters and backprops with random gradients:
net.zero_grad()
out.backward(torch.randn(1, 10))


In [18]:
output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 39.0337
[torch.FloatTensor of size 1]



In [19]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [20]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
 0.1454
-0.1415
 0.0035
-0.0349
 0.0170
-0.0095
[torch.FloatTensor of size 6]



In [21]:
#update weights
#weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [23]:
import torch.optim as optim

#target = Variable(torch.arange(1, 11))  # a dummy target, for example
#target = target.view(1, -1)  # make it the same shape as output


# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
print ("done")

done
